Our NLP project consists of creating a financial tool. This tool would read any company's financial statement and provide important ratios and some recommendations.

In [125]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import balance_reader as br

The first step we took was to import the necessary libraries. We also imported our balance reader tool, which is saved as a .py document.

In [159]:
df_1 = br.find_variables('datafast_21.pdf')
comments_df = pd.read_csv('updated_outcomes_with_comments_vf.csv')

The second step we took was to upload any financial statement as a PDF document. Using our balance reader tool, we can read it and extract the necessary variables required for calculating the needed ratios. Additionally, we uploaded our CSV document that contains all the possible recommendations.

In [160]:
df_copy = df_1.copy()

In [161]:
comments_df ['S_liquidity'] = comments_df ['S_liquidity'].str.lower()
comments_df ['S_Leverage_Ratio'] = comments_df ['S_Leverage_Ratio'].str.lower()
comments_df ['S_ROE'] = comments_df ['S_ROE'].str.lower()
comments_df ['S_Net Revenue Percentage Delta'] = comments_df ['S_Net Revenue Percentage Delta'].str.lower()

In [162]:
df_copy 

,Company Name,Year,Variable Name,Value
0,DATAFAST S.A.,2021,Current Assets,"5,813,989"
1,DATAFAST S.A.,2020,Current Assets,"4,701,763"
2,DATAFAST S.A.,2021,Assets,"17,085,078"
3,DATAFAST S.A.,2020,Assets,"15,965,292"
4,DATAFAST S.A.,2021,Equity,"10,095,148"
5,DATAFAST S.A.,2020,Equity,"9,091,400"
6,DATAFAST S.A.,2021,Current Liabilities,"5,473,162"
7,DATAFAST S.A.,2020,Current Liabilities,"5,465,290"
8,DATAFAST S.A.,2021,Liabilities,"6,989,930"
9,DATAFAST S.A.,2020,Liabilities,"6,873,892"


This is how our balance reader provides us with the extracted data.

In [163]:
# Assuming data_pivoted is your DataFrame
#Convert relevant columns to numeric

#convert Value to string and remove points and commas
df_copy['Value'] = df_copy['Value'].astype(str).str.replace('.', '').str.replace(',', '')

#convert Value to numeric
df_copy['Value'] = pd.to_numeric(df_copy['Value'], errors='coerce')


In [164]:
df_copy

,Company Name,Year,Variable Name,Value
0,DATAFAST S.A.,2021,Current Assets,5813989
1,DATAFAST S.A.,2020,Current Assets,4701763
2,DATAFAST S.A.,2021,Assets,17085078
3,DATAFAST S.A.,2020,Assets,15965292
4,DATAFAST S.A.,2021,Equity,10095148
5,DATAFAST S.A.,2020,Equity,9091400
6,DATAFAST S.A.,2021,Current Liabilities,5473162
7,DATAFAST S.A.,2020,Current Liabilities,5465290
8,DATAFAST S.A.,2021,Liabilities,6989930
9,DATAFAST S.A.,2020,Liabilities,6873892


In [165]:
data_pivoted = df_copy.pivot_table(index='Year', columns='Variable Name', values='Value', aggfunc='first')
data_pivoted

Variable Name,Assets,Current Assets,Current Liabilities,Equity,Liabilities,Profit
Year,,,,,,
2020,15965292,4701763,5465290,9091400,6873892,673513
2021,17085078,5813989,5473162,10095148,6989930,1064238


After some transformations, we finally obtain this dataframe, which is formatted as we need the data. 

In [166]:
# Now calculate the ratios
data_pivoted['Liquidity Ratio'] = data_pivoted['Current Assets'] / data_pivoted['Current Liabilities']
data_pivoted['Leverage Ratio'] = data_pivoted['Liabilities'] / data_pivoted['Equity']
data_pivoted['ROE'] = data_pivoted['Profit'] / data_pivoted['Equity']
data_pivoted['Net Revenue Percentage Delta'] = data_pivoted['Profit'].pct_change() * 100

In [167]:
data_pivoted

Variable Name,Assets,Current Assets,Current Liabilities,Equity,Liabilities,Profit,Liquidity Ratio,Leverage Ratio,ROE,Net Revenue Percentage Delta
Year,,,,,,,,,,
2020,15965292,4701763,5465290,9091400,6873892,673513,0.860295,0.756087,0.074082,NaN
2021,17085078,5813989,5473162,10095148,6989930,1064238,1.062272,0.692405,0.105421,58.012986


We added the columns that calculate the ratios to the dataframe.

# Calssification Rules

We created ranges for each ratio to classify the data, based on a traffic light system: RED indicates poor performance, YELLOW is satisfactory, and GREEN signifies good performance. This classification is determined by the actual ratios obtained from the financial statements of each company.

In [168]:
data_pivoted['S_liquidity'] = data_pivoted['Liquidity Ratio'].apply(lambda x: 'green' if x > 1 else ('red' if x < 0.5 else 'yellow'))
data_pivoted['S_Leverage_Ratio'] = data_pivoted['Leverage Ratio'].apply(lambda x: 'green' if x > 0.8 else ('red' if x < 0.2 else 'yellow'))
data_pivoted['S_ROE'] = data_pivoted['ROE'].apply(lambda x: 'green' if x > 0.03 else ('red' if x < 0 else 'yellow'))
data_pivoted['S_Net Revenue Percentage Delta'] = data_pivoted['Net Revenue Percentage Delta'].apply(lambda x: 'green' if x > 0.01 else ('red' if x < 0 else 'yellow'))

In [169]:
data_pivoted

Variable Name,Assets,Current Assets,Current Liabilities,Equity,Liabilities,Profit,Liquidity Ratio,Leverage Ratio,ROE,Net Revenue Percentage Delta,S_liquidity,S_Leverage_Ratio,S_ROE,S_Net Revenue Percentage Delta
Year,,,,,,,,,,,,,,
2020,15965292,4701763,5465290,9091400,6873892,673513,0.860295,0.756087,0.074082,NaN,yellow,yellow,green,yellow
2021,17085078,5813989,5473162,10095148,6989930,1064238,1.062272,0.692405,0.105421,58.012986,green,yellow,green,green


# Coments

Finally, we merged the obtained dataframe with the CSV comments. There are more than 80 different possible outputs, each specifically associated with a particular group of comments.

In [170]:
merged_df = pd.merge(data_pivoted, comments_df , on=['S_liquidity', 'S_Leverage_Ratio', 'S_ROE', 'S_Net Revenue Percentage Delta'], how='left')

In [171]:
merged_df

,Assets,Current Assets,Current Liabilities,Equity,Liabilities,Profit,Liquidity Ratio,Leverage Ratio,ROE,Net Revenue Percentage Delta,S_liquidity,S_Leverage_Ratio,S_ROE,S_Net Revenue Percentage Delta,Coments
0,15965292,4701763,5465290,9091400,6873892,673513,0.860295,0.756087,0.074082,NaN,yellow,yellow,green,yellow,Good job! Try to use your liquidity surplus to...
1,17085078,5813989,5473162,10095148,6989930,1064238,1.062272,0.692405,0.105421,58.012986,green,yellow,green,green,Fantastic! Your company is soaring. Keep up th...


In [172]:
# create a new column as liquidity comments which is based on column S_liquidity -	red	-Watch out! Your liquidity is too low. You don't even have half the money to cover your short-term responsibilities,yellow -	You've got some cash to handle your short-term debts, but you're not fully covered. It's time to come up with a strategy to boost your liquidity,green -Congratulations! You've covered all your short-term liabilities. Great job!
merged_df['Liquidity Comments'] = merged_df['S_liquidity'].apply(lambda x: 'Watch out! Your liquidity is too low. You don\'t even have half the money to cover your short-term responsibilities' if x == 'red' else ('You\'ve got some cash to handle your short-term debts, but you\'re not fully covered. It\'s time to come up with a strategy to boost your liquidity' if x == 'yellow' else 'Congratulations! You\'ve covered all your short-term liabilities. Great job!'))							


In [173]:
merged_df

,Assets,Current Assets,Current Liabilities,Equity,Liabilities,Profit,Liquidity Ratio,Leverage Ratio,ROE,Net Revenue Percentage Delta,S_liquidity,S_Leverage_Ratio,S_ROE,S_Net Revenue Percentage Delta,Coments,Liquidity Comments
0,15965292,4701763,5465290,9091400,6873892,673513,0.860295,0.756087,0.074082,NaN,yellow,yellow,green,yellow,Good job! Try to use your liquidity surplus to...,You've got some cash to handle your short-term...
1,17085078,5813989,5473162,10095148,6989930,1064238,1.062272,0.692405,0.105421,58.012986,green,yellow,green,green,Fantastic! Your company is soaring. Keep up th...,Congratulations! You've covered all your short...


In [174]:
## create a new column as 'ROE comments' which is based on column S_ROE : 	ROE 	RED		Your profit is in the red, meaning you're not adding value for your shareholders. It's time to rethink your strategy.YELLOW 		Even though you're making some money, there's still work to be done. Your shareholders will be happier with improvements.GREEN 		Amazing job! Your returns look great, but remember, there's always room for improvement.
merged_df['ROE comments'] = merged_df['S_ROE'].apply(lambda x: 'Your profit is in the red, meaning you\'re not adding value for your shareholders. It\'s time to rethink your strategy.' if x == 'red' else ('Even though you\'re making some money, there\'s still work to be done. Your shareholders will be happier with improvements.' if x == 'yellow' else 'Amazing job! Your returns look great, but remember, there\'s always room for improvement.'))

#create a new column as 'Leverage Ratio comments' which is based on column S_Leverage_Ratio : RED		Be careful! Your debt is too high. Decrease it as soon as possible (ASAP).YELLOW 	Your debt is high, but you're still within an acceptable range. Try not to increase it further.GREEN 		Congratulations! You're in a very comfortable position with your debt.
merged_df['Leverage Ratio comments'] = merged_df['S_Leverage_Ratio'].apply(lambda x: 'Be careful! Your debt is too high. Decrease it as soon as possible (ASAP).' if x == 'red' else ('Your debt is high, but you\'re still within an acceptable range. Try not to increase it further.' if x == 'yellow' else 'Congratulations! You\'re in a very comfortable position with your debt.'))

#create a new column as 'Net Revenue Percentage Delta Comments' which is based on column S_Net Revenue Percentage Delta : 	LEVERAGE RATIO 	RED		Be careful! Your debt is too high. Decrease it as soon as possible (ASAP).YELLOW 		Your debt is high, but you're still within an acceptable range. Try not to increase it further.GREEN 		Congratulations! You're in a very comfortable position with your debt.
merged_df['Net Revenue Percentage Delta Comments'] = merged_df['S_Net Revenue Percentage Delta'].apply(lambda x: 'Your profit is in the red, meaning you\'re not adding value for your shareholders. It\'s time to rethink your strategy.' if x == 'red' else ('Even though you\'re making some money, there\'s still work to be done. Your shareholders will be happier with improvements.' if x == 'yellow' else 'Amazing job! Your returns look great, but remember, there\'s always room for improvement.'))




This is our final dataframe containing all the ratios and recommendations.

In [175]:
#see all columns
pd.set_option('display.max_columns', None)
merged_df

,Assets,Current Assets,Current Liabilities,Equity,Liabilities,Profit,Liquidity Ratio,Leverage Ratio,ROE,Net Revenue Percentage Delta,S_liquidity,S_Leverage_Ratio,S_ROE,S_Net Revenue Percentage Delta,Coments,Liquidity Comments,ROE comments,Leverage Ratio comments,Net Revenue Percentage Delta Comments
0,15965292,4701763,5465290,9091400,6873892,673513,0.860295,0.756087,0.074082,NaN,yellow,yellow,green,yellow,Good job! Try to use your liquidity surplus to...,You've got some cash to handle your short-term...,"Amazing job! Your returns look great, but reme...","Your debt is high, but you're still within an ...","Even though you're making some money, there's ..."
1,17085078,5813989,5473162,10095148,6989930,1064238,1.062272,0.692405,0.105421,58.012986,green,yellow,green,green,Fantastic! Your company is soaring. Keep up th...,Congratulations! You've covered all your short...,"Amazing job! Your returns look great, but reme...","Your debt is high, but you're still within an ...","Amazing job! Your returns look great, but reme..."
